# Korean LLM (Large Language Model) Serving on SageMaker with AWS Large Model Container DLC
---

- LLM GitHub: https://github.com/nlpai-lab/KULLM
- Hugging Face model hub: https://huggingface.co/nlpai-lab/kullm-polyglot-5.8b-v2
- [AWS Blog: Deploy large models on Amazon SageMaker using DJLServing and DeepSpeed model parallel inference](https://aws.amazon.com/ko/blogs/machine-learning/deploy-large-models-on-amazon-sagemaker-using-djlserving-and-deepspeed-model-parallel-inference)

## Overview 

기존에 SageMaker 호스팅 인스턴스에서 모델 서빙 시에는 모델 아티팩트를 model.tar.gz 형식으로 아카이브하여 압축하였습니다. 이 방법은 소규모 모델에는 효과적이지만, 수백억~수천억 개의 파라미터를 가진 파운데이션 모델에는 적합하지 않습니다. (참조: https://docs.aws.amazon.com/sagemaker/latest/dg/large-model-inference-uncompressed.html)
 
본 핸즈온에서는 PEFT로 훈련된 추가 모델 파라미터를 베이스 모델 파라미터에 병합하여 SageMaker의 LMI(Large Model Inference) 컨테이너 환경에 모델을 배포하는 법을 알아봅니다.

In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('./utils')
sys.path.append('./templates')

from common_lib import check_packages
check_packages()

%store -r train_job_name

try:
    train_job_name
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] 3번 모듈 노트북(3_sm-train-lora.ipynb)을 먼저 실행해 주세요.")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

In [ ]:
import os
import json
import boto3
import torch
import sagemaker, boto3, jinja2
from sagemaker import get_execution_role
from sagemaker.model import Model
from sagemaker.estimator import Estimator
from sagemaker import serializers, deserializers
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer

role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # bucket to house artifacts
region = sess._region_name 
account_id = sess.account_id()  # account_id of the current SageMaker Studio environment
s3_client = boto3.client("s3")  # client to intreract with S3 API
sm_client = boto3.client("sagemaker")  # client to intreract with SageMaker
smr_client = boto3.client("sagemaker-runtime")  # client to intreract with SageMaker Endpoints

estimator = Estimator.attach(train_job_name)

<br>

## 1. Merge PEFT model
---

In [ ]:
local_peft_model_dir = 'model_from_sagemaker'

if not os.path.exists(local_peft_model_dir):
    os.makedirs(local_peft_model_dir)

!aws s3 cp {estimator.model_data} {local_peft_model_dir}/model.tar.gz
!tar -xzf {local_peft_model_dir}/model.tar.gz -C {local_peft_model_dir}
!rm {local_peft_model_dir}/model.tar.gz

In [ ]:
def modify_base_model_path(peft_model_dir, base_model_path):
    with open(f"{peft_model_dir}/adapter_config.json", "r") as jsonfile:
        data = json.load(jsonfile)
    data["base_model_name_or_path"] = base_model_path
    with open(f"{peft_model_dir}/adapter_config.json", "w") as jsonfile:
        json.dump(data, jsonfile)

def byte_transform(byte_size, to):
    a = {'k': 1, 'm': 2, 'g': 3, 't': 4, 'p': 5, 'e': 6}
    r = float(byte_size)
    for i in range(a[to]):
        r = r / 1024
    return round(r, 4)

size = 0
for ele in os.scandir(local_peft_model_dir):
    size += os.path.getsize(ele)
gb_size = byte_transform(size, 'g')

In [ ]:
if gb_size < 0.5:
    base_model_path = "/home/ec2-user/SageMaker/models/kullm-polyglot-12-8b-v2"
    modify_base_model_path(local_peft_model_dir, base_model_path)
    
    local_model_dir = "model_from_sagemaker_merged"    
    print(f'Save merged model: {local_model_dir}')    
    os.makedirs(local_model_dir, exist_ok=True)
    model = AutoPeftModelForCausalLM.from_pretrained(local_peft_model_dir, low_cpu_mem_usage=True, torch_dtype=torch.bfloat16)
    merged_model = model.merge_and_unload()
    merged_model.save_pretrained(local_model_dir, safe_serialization=True)
    
    tokenizer = AutoTokenizer.from_pretrained(local_peft_model_dir)
    tokenizer.save_pretrained(local_model_dir)
else:
    local_model_dir = local_peft_model_dir

<br>

## 2. Upload LLM model to S3
---

In [ ]:
model_prefix = "kkulm-qlora-12-8B"
model_tar_dir = f"{os.getcwd()}/model_from_sagemaker_merged"

bucket_prefix = 'ko-llms/serving'    
s3_code_prefix = f"{bucket_prefix}/{model_prefix}/code"  # folder within bucket where code artifact will go
s3_model_prefix = f"{bucket_prefix}/{model_prefix}/model"  # folder where model checkpoint will go
s3_model_artifact = f"s3://{bucket}/{s3_model_prefix}"

print(f"S3 code prefix \n {s3_code_prefix}")
print(f"S3 model prefix: \n {s3_model_prefix}")
print(f"S3 model artifact path: \n {s3_model_artifact}")

In [ ]:
%%bash
aws configure set default.s3.max_concurrent_requests 100
aws configure set default.s3.max_queue_size 10000
aws configure set default.s3.multipart_threshold 1GB
aws configure set default.s3.multipart_chunksize 64MB

In [ ]:
!aws s3 sync {model_tar_dir} {s3_model_artifact}
print(f"Model uploaded to --- > {s3_model_artifact}")
print(f"We will set option.s3url={s3_model_artifact}")

In [ ]:
src_path = f"serving_src/{model_prefix}"
!rm -rf {src_path}
os.makedirs(src_path, exist_ok=True)

In [ ]:
%%writefile {src_path}/serving.properties
option.s3url={{s3url}}

engine=DeepSpeed

# passing extra options to model.py or built-in handler
job_queue_size=100
batch_size=1
max_batch_delay=1
max_idle_time=60

# Built-in entrypoint
#option.entryPoint=djl_python.deepspeed

# Hugging Face model id
#option.model_id=EleutherAI/gpt-j-6B

# defines custom environment variables
#env=SERVING_NUMBER_OF_NETTY_THREADS=2

# Allows to load DeepSpeed workers in parallel
option.parallel_loading=true

# specify tensor parallel degree (number of partitions)
option.tensor_parallel_degree=4

# specify per model timeout
option.model_loading_timeout=600
#option.predict_timeout=240

# mark the model as failure after python process crashing 10 times
retry_threshold=0

option.task=text-generation

In [ ]:
from pathlib import Path
jinja_env = jinja2.Environment()  
# we plug in the appropriate model location into our `serving.properties` file based on the region in which this notebook is running
template = jinja_env.from_string(Path(f"{src_path}/serving.properties").open().read())
Path(f"{src_path}/serving.properties").open("w").write(template.render(s3url=s3_model_artifact))
!pygmentize {src_path}/serving.properties | cat -n

In [ ]:
%%writefile {src_path}/model.py
from djl_python import Input, Output
import os
import deepspeed
import torch
import logging
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
from transformers import GPTNeoXLayer

predictor = None

def get_model(properties):
    
    tp_degree = properties["tensor_parallel_degree"]
    model_location = properties["model_dir"]
    if "model_id" in properties:
        model_location = properties["model_id"]
    task = properties["task"]
    
    logging.info(f"Loading model in {model_location}")    
    local_rank = int(os.getenv("LOCAL_RANK", "0"))

    tokenizer = AutoTokenizer.from_pretrained(model_location)

    model = AutoModelForCausalLM.from_pretrained(
        model_location,
        torch_dtype=torch.float16,
        low_cpu_mem_usage=True,
    )
    
    model.requires_grad_(False)
    model.eval()
    
    ds_config = {
        "tensor_parallel": {"tp_size": tp_degree},
        "dtype": model.dtype,
        "injection_policy": {
            GPTNeoXLayer:('attention.dense', 'mlp.dense_4h_to_h')
        }
    }
    logging.info(f"Starting DeepSpeed init with TP={tp_degree}")        
    model = deepspeed.init_inference(model, ds_config)  
    
    generator = pipeline(
        task=task, model=model, tokenizer=tokenizer, device=local_rank
    )
    # https://huggingface.co/docs/hub/models-tasks
    return generator
    
def handle(inputs: Input) -> None:
    """
    inputs: Contains the configurations from serving.properties
    """    
    global predictor
    if not predictor:
        predictor = get_model(inputs.get_properties())

    if inputs.is_empty():
        # Model server makes an empty call to warmup the model on startup
        logging.info("is_empty")
        return None

    data = inputs.get_as_json() #inputs.get_as_string()
    logging.info("data:", data)
    
    input_prompt, params = data["inputs"], data["parameters"]
    result = predictor(input_prompt, **params)
    logging.info("result:", result)

    return Output().add_as_json(result) #Output().add(result)    

In [ ]:
!rm -rf model.tar.gz
!tar czvf model.tar.gz -C {src_path} .

In [ ]:
s3_code_artifact = sess.upload_data("model.tar.gz", bucket, s3_code_prefix)
print(f"S3 Code or Model tar ball uploaded to --- > {s3_code_artifact}")
!rm -rf model.tar.gz

<br>

## 3. Serve LLM Model on SageMaker
---
### Create SageMaker Model

SageMaker 엔드포인트 생성 매개변수 VolumeSizeInGB를 지정할 때 마운트되는 Amazon EBS(Amazon Elastic Block Store) 볼륨에 /tmp를 매핑하기 때문에 컨테이너는 인스턴스의 `/tmp` 공간에 모델을 다운로드합니다. 이때 s5cmd (https://github.com/peak/s5cmd) 를 활용하므로 대용량 모델을 빠르게 다운로드할 수 있습니다.
볼륨 인스턴스와 함께 미리 빌드되어 제공되는 p4dn과 같은 인스턴스의 경우 컨테이너의 `/tmp`를 계속 활용할 수 있습니다. 

In [ ]:
from sagemaker.utils import name_from_base
from sagemaker import image_uris

img_uri = image_uris.retrieve(framework="djl-deepspeed", region=region, version="0.23.0")
model_name = name_from_base(f"{model_prefix}")
print(model_name)

model_response = sm_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer={"Image": img_uri, "ModelDataUrl": s3_code_artifact},
)
model_arn = model_response["ModelArn"]
print(f"Created Model: {model_arn}")

### Create SageMaker Endpoint

In [ ]:
endpoint_config_name = f"{model_name}-config"
endpoint_name = f"{model_name}-endpoint"
variant_name = "variant1"
instance_type = "ml.g5.12xlarge"
initial_instance_count = 1

prod_variants = [
    {
        "VariantName": "variant1",
        "ModelName": model_name,
        "InstanceType": instance_type,
        "InitialInstanceCount": initial_instance_count,
        # "ModelDataDownloadTimeoutInSeconds": 2400,
        "ContainerStartupHealthCheckTimeoutInSeconds": 600,
    }
]

endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=prod_variants
)

endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name
)
print(f"Created Endpoint: {endpoint_response['EndpointArn']}")

엔드포인트가 생성되는 동안 아래의 문서를 같이 확인해 보세요.
- https://docs.aws.amazon.com/sagemaker/latest/dg/large-model-inference-dlc.html

In [ ]:
from IPython.display import display, HTML
def make_console_link(region, endpoint_name, task='[SageMaker LLM Serving]'):
    endpoint_link = f'<b> {task} <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={region}#/endpoints/{endpoint_name}">Check Endpoint Status</a></b>'   
    return endpoint_link

endpoint_link = make_console_link(region, endpoint_name)
display(HTML(endpoint_link))

In [ ]:
%%time 
from inference_lib import describe_endpoint, Prompter
describe_endpoint(endpoint_name)         

<br>

## 4. Inference
---

엔드포인트를 호출할 때 이 텍스트를 JSON 페이로드 내에 제공해야 합니다. 이 JSON 페이로드에는 length, sampling strategy, output token sequence restrictions을 제어하는 데 도움이 되는 원하는 추론 매개변수가 포함될 수 있습니다. 허깅페이스 트랜스포머 transformers 라이브러리에는 [사용 가능한 페이로드 매개변수](https://huggingface.co/docs/transformers/main_classes/text_generation#transformers.GenerationConfig)의 전체 목록이 정의되어 있지만, 중요한 페이로드 매개변수는 다음과 같이 정의되어 있습니다:

* **do_sample (`bool`)** – logits sampling 활성화
* **max_new_tokens (`int`)** – 생성된 토큰의 최대 수
* **best_of (`int`)** – best_of 개의 시퀀스를 생성하고 가장 높은 토큰 로그 확률이 있는 경우 반환
* **repetition_penalty (`float`)** – 반복 패널티에 대한 파라미터, 1.0은 패널티가 없음을 의미하여 Greedy 서치와 동일, 커질수록 다양한 결과를 얻을 수 있으며, 자세한 사항은 [this paper](https://arxiv.org/pdf/1909.05858.pdf)을 참고
* **return_full_text (`bool`)** – 생성된 텍스트에 프롬프트를 추가할지 여부
* **seed (`int`)** – Random sampling seed
* **stop_sequences (`List[str]`)** – `stop_sequences` 가 생성되면 토큰 생성을 중지
* **temperature (`float`)** – logits 분포 모듈화에 사용되는 값
* **top_k (`int`)** – 상위 K개 만큼 가장 높은 확률 어휘 토큰의 수
* **top_p (`float`)** – 1 보다 작게 설정하게 되며, 상위부터 정렬했을 때 가능한 토큰들의 확률을 합산하여 `top_p` 이상의 가장 작은 집합을 유지
* **truncate (`int`)** – 입력 토큰을 지정된 크기로 잘라냄
* **typical_p (`float`)** – typical Decoding 양으로, 자세한 사항은 [Typical Decoding for Natural Language Generation](https://arxiv.org/abs/2202.00666)을 참고
* **watermark (`bool`)** –  [A Watermark for Large Language Models](https://arxiv.org/abs/2301.10226)가 Watermarking
* **decoder_input_details (`bool`)** – decoder input token logprobs와 ids를 반환

In [ ]:
params = {
    "do_sample": True,
    "max_new_tokens": 128,
    "temperature": 0.7,
    "top_p": 0.9,
    "return_full_text": False,
    "repetition_penalty": 1.1,
    "presence_penalty": None,
    "eos_token_id": 2,
}

In [ ]:
import json
from inference_lib import KoLLMSageMakerEndpoint, Prompter
ep = KoLLMSageMakerEndpoint(endpoint_name)

In [ ]:

instruction = "다음 글을 알기 쉽게 요약해 주세요."
context = """
대규모 언어 모델(LLM; Large Language Models) 분야의 발전과 LLM이 가치 있는 인사이트를 제공하는 문제 세트 수가 계속 증가하고 있다는 소식을 들어보셨을 겁니다. 
대규모 데이터 세트와 여러 작업을 통해 훈련된 대규모 모델은 훈련되지 않은 특정 작업에도 잘 일반화됩니다. 이러한 모델을 파운데이션 모델(foundation model)이라고 하며, 스탠포드 HAI 연구소(Stanford Institute for Human-Centered Artificial Intelligence)에서 처음 대중화한 용어입니다. 
이러한 파운데이션 모델은 프롬프트 엔지니어링(prompt engineering) 기법의 도움으로 잘 활용할 수 있지만, 유스케이스가 도메인에 매우 특화되어 있거나 작업의 종류가 매우 다양하여 모델을 추가로 커스터마이징해야 하는 경우가 많습니다. 
특정 도메인이나 작업에 대한 대규모 모델의 성능을 개선하기 위한 한 가지 접근 방식은 더 작은 작업별 데이터 세트로 모델을 추가로 훈련하는 것입니다. 파인 튜닝(fine-tuning)으로 알려진 이 접근 방식은 LLM의 정확도를 성공적으로 개선하지만, 모든 모델 가중치를 수정해야 합니다. 
파인 튜닝 데이터 세트 크기가 훨씬 작기 때문에 사전 훈련(pre-training)하는 것 보다 훨씬 빠르지만 여전히 상당한 컴퓨팅 성능과 메모리가 필요합니다. 
파인 튜닝은 원본 모델의 모든 파라미터 가중치를 수정하므로 비용이 많이 들고 원본 모델과 동일한 크기의 모델을 생성하므로 스토리지 용량에도 부담이 됩니다.
"""
payload = ep.get_payload(instruction, context, params)

In [ ]:
%%time
generated_text = ep.infer(payload, verbose=True)

<br>

## 5. Clean Up
---

In [ ]:
!rm -rf {local_peft_model_dir} {local_model_dir} serving_src

In [ ]:
# - Delete the end point
sm_client.delete_endpoint(EndpointName=endpoint_name)
# - In case the end point failed we still want to delete the model
sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
sm_client.delete_model(ModelName=model_name)